<img style="float: left; margin:0px 15px 15px 0px; width:120px" src="https://www.orfeo-toolbox.org/wp-content/uploads/2016/03/logo-orfeo-toolbox.png">

# OTB Guided Tour - JURSE 2019 Vannes France - May 21th
## Yannick TANGUY and David YOUSSEFI (CNES, French Space Agency)

<br>

<b> Press <span style="color:black;background:yellow">SHIFT+ENTER</span> to execute the notebook interactively cell by cell </b></div>

## Step 4 : Filter shapefile

### Polygonize with GDAL

In [1]:
import os
import subprocess

import ogr_io

# Data directory
DATA_DIR = "data"

# Output directory
OUTPUT_DIR = "output"

# Input / Output filenames
ndwi_thres_fname = os.path.join(OUTPUT_DIR, "ndwi_threshold30.tif")
watermask_fname = os.path.join(OUTPUT_DIR, "watermask.sqlite")
envelope_fname = os.path.join(OUTPUT_DIR, "morbihan.sqlite")
results_fname = os.path.join(OUTPUT_DIR, "results.sqlite")
geojson_fname = os.path.join(OUTPUT_DIR, "geojson.json")

# Convert the TIF file in a shapefile with polygons
try:
    subprocess.call(["gdal_polygonize.py", ndwi_thres_fname,"-f", "SQLite", watermask_fname])
    print ("Converting ",ndwi_thres_fname, " into a shapefile ", watermask_fname)
except:
    print("Error during conversion")
    exit(-1)

Converting  output/ndwi_threshold30.tif  into a shapefile  output/watermask.sqlite


In [2]:
# Read shapefile with ogr
watermask = ogr_io.openToRead(watermask_fname)
envelope = ogr_io.openToRead(envelope_fname)

### Crop Islands

Crop the "watermask" shapefile with envelope shape (~ Morbihan gulf) and also filter the smallest areas and the two biggest (main land and sea)


In [3]:
from osgeo import ogr, osr

# Save extent to a new Shapefile
outDriver = ogr.GetDriverByName("SQLite")

# Remove output shapefile if it already exists
if os.path.exists(results_fname):
    outDriver.DeleteDataSource(results_fname)

# create the spatial reference, WGS84
srs = osr.SpatialReference()
srs.ImportFromEPSG(32630)

# Create the output shapefile
outDataSource = outDriver.CreateDataSource(results_fname)
outLayer = outDataSource.CreateLayer("islands", srs, geom_type=ogr.wkbPolygon)

# Collect all islands inside the envelope
geomcol = ogr.Geometry(ogr.wkbGeometryCollection)
inLayer = watermask.GetLayer()
layer_envelope = envelope.GetLayer()

for shape in layer_envelope:
    inLayer.SetSpatialFilter(shape.GetGeometryRef())
    print("Nb features (before filtering): ",inLayer.GetFeatureCount())
    for feature in inLayer:
        outLayer.CreateFeature(feature)

featureDefn = outLayer.GetLayerDefn()

areaMax = 0
index = 1
biggestFeature = 0
for feature in outLayer:
    area = feature.GetGeometryRef().GetArea()
    # Our features contain : 
    # - some land (because the cropping shape is not very precise
    # - the sea !
    # - some very small submarine dark areas : these are supposely rocks
    # --> we eliminate every feature larger than 4 km² ("ile aux moines" area) 
    # and also smaller than 10000 m² (1 Ha)
    if (area > 4000000.0):
        #print("Del too large feature area = ",str(area))
        outLayer.DeleteFeature(feature.GetFID())
    if (area < 10000.0):
        #print("Del too small feature area = ",str(area))
        outLayer.DeleteFeature(feature.GetFID())

print ("Nb islands (after filtering): ", outLayer.GetFeatureCount())
    
# unload the driver -> writes the file
outDriver = None

Nb features (before filtering):  344
Nb islands (after filtering):  45


In [7]:
import rasterio
from glob import glob
import display_api
import json

# Convert sqlite to geojson EPSG:4326
try:
    subprocess.call(["ogr2ogr", "-f", "GeoJSON", geojson_fname, results_fname, "-t_srs", "EPSG:4326"])
    print ("Converting sqlite to geojson EPSG:4326")
except:
    print("Error during conversion")
    exit(-1)
    
with open(geojson_fname, 'r') as f:
    fc = json.load(f)

DATE = "20180805"
raster = rasterio.open(glob(os.path.join(DATA_DIR, "SENTINEL2?_{0}-*.tif".format(DATE)))[0])
m, dc = display_api.rasters_on_map([raster], OUTPUT_DIR, [DATE], geojson_data=fc)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'attribution': 'Map data (c) <a href…